In [1]:
class HashTable:
    def __init__(self, m=10):    # a default array length of 10
        self.inArray = [LinkedList() for i in range(m)] 
        self.size = 0
        self.threshold = 0.75    # a default threshold of 0.75

    def hash(self, d):
        return d % len(self.inArray)
           
    def add(self, d):
        i = self.hash(d)
        self.inArray[i].insert(0,d)
        self.size += 1
        # condition that must be respected: self.size <= self.threshold * len(self.inArray)
        if self.size > self.threshold*len(self.inArray): self._resizeUp()
        
    def search(self, d):
        i = self.hash(d)
        if self.inArray[i].search(d) == -1: return False
        return True

    def remove(self, d):
        i = self.hash(d)
        if self.inArray[i].removeVal(d):
            self.size -= 1

    def _resizeUp(self):
        oldArray = self.inArray
        self.inArray = [LinkedList() for _ in range(2*len(oldArray))]
        for l in oldArray:
            while l.length > 0:
                d = l.remove(0)
                self.inArray[self.hash(d)].insert(0,d)            
            
    # these are to make our life easier in adding/removing many elements
    def addAll(self, A):
        for i in range(len(A)):
            self.add(A[i])

    def removeAll(self, A):
        for i in range(len(A)):
            self.remove(A[i])

            
def myprint(h):
    for i in range(len(h.inArray)):
        print("pos",i,":",h.inArray[i])
    print()
        
A = [0, 34, 23, 4, 5, -2, -13, 42, 45, 10, 56, 90, 99, 10, 2, 2]
h = HashTable()
print(0,": hashtable (size "+str(h.size)+"):\n"); myprint(h)

# next add 7 elements
h.addAll(A[:7]) 
print(1,": hashtable (size "+str(h.size)+"):\n"); myprint(h)

# add 1 more, so 8 elements now > 0.75*10 -- need to resize up
h.add(A[7]) 
print(2,": hashtable (size "+str(h.size)+"):\n"); myprint(h)

# add a few more
h.addAll(A[8:]) 
print(3,": hashtable (size "+str(h.size)+"):\n"); myprint(h)

import random
# and now remove all but one
random.shuffle(A)
h.removeAll(A[1:]) 
print(4,": hashtable (size "+str(h.size)+"):\n"); myprint(h)
print("last element",A[0])


NameError: name 'LinkedList' is not defined

In [2]:
def prehash(s):
#     return len(s)
    return ord(s[-1])-96

def prehash(s, b=31):
    h = 0; m = 1
    for c in s:
        h += m*(ord(c)-96)
        m *= b
    return h

prehash("abc"), prehash("foo")

(2946, 14886)

In [3]:
print("0:",hash(0),hash(1),hash(42))

# collisions
print("1:",hash(2**61-2),hash(2**61-1),hash(2**61))
print("2:",hash(0),hash(-1),hash(-2),hash(-3))

i1 = 123; i2 = 123; i3 = 1234
print("3:",hash(i1),hash(i2),hash(i3))

st1 = "123"; st2 = "123"; st3 = "1234"
print("4:",hash(st1),hash(st2),hash(st3))

l1 = LinkedList(); l2 = LinkedList()
print("5:",hash(l1),hash(l2))

print("6:",hash(True),hash(False))

0: 0 1 42
1: 2305843009213693950 0 1
2: 0 -2 -2 -3
3: 123 123 1234
4: -8428224885829604265 -8428224885829604265 -8619410607401247371


NameError: name 'LinkedList' is not defined

In [4]:
m = {}
print("0:",m, type(m),"\n")

# add (key,value) pair to dictionary 
m["foo"] = "a standard name for variables" # add the pair ("foo","a standard name for variables")
m["bar"] = "another standard name for variables"
print("1:",m,"\n")

# membership in dictionary
print("2:","foo" in m, "fo" in m,"\n") 

# remove key (and value) from dictionary
del(m["foo"])
print("3:","foo" in m, "fo" in m,"\n") 

# update key value in dictionary
m["algorithms"] = "are great!"
print("4:",m,"\n")
m["algorithms"] = "go well with data structures"
print("5:",m,"\n")

# iteration on keys
print("6:")
for x in m:
    print(" -",x,":",m[x])

0: {} <class 'dict'> 

1: {'foo': 'a standard name for variables', 'bar': 'another standard name for variables'} 

2: True False 

3: False False 

4: {'bar': 'another standard name for variables', 'algorithms': 'are great!'} 

5: {'bar': 'another standard name for variables', 'algorithms': 'go well with data structures'} 

6:
 - bar : another standard name for variables
 - algorithms : go well with data structures


## Hash Maps

We next implement hash maps, which are hash tables that store objects 
that are pairs of the form: (key,value). Each key in the hash map has at most one 
corresponding value. Hashing of such pairs (in order to search for them, remove them, etc.) is done with respect
to their key. That is, to search if there is a pair with key 42, we need to hash 42 to retrieve 
an index, and then search the linked list in the corresponding entry of the internal array for 
a pair with key 42. 

To store pairs, we will use the following class, where note that we take two pairs to be 
equal just if their keys are equal. 

    class KVPair:
        def __init__(self, k, v):
            self.key = k
            self.val = v
        def __eq__(self, other):
            return self.key == other.key

We need to adapt the implementation of `HashTable` into a class `HashMap` with functions:

- `def search(self,k)` : searches for a pair with key `k` in the hash map and returns 
`True` if it finds one, otherwise `False`.
- `def remove(self,k)` : searches for a pair with key `k` in the hash map and removes 
it if it finds its, otherwise it leaves the hash map unchanged.
- `def get(self,k)` : returns the value `v` with which the key `k` is paired in the hash 
map (i.e. such that `(k,v)` is in the hash map). If there is no such pair, returns `None`.
- `def put(self,k,v)` : adds the pair `(k,v)` in the hash map. If there is already a pair 
with key `k` in the map, then its value is updated to `v`.

Our hash maps work with keys and values of arbitrary type. For example, executing the following code:

    h = HashMap()
    h.put(10,12); print(h, h.size, h.search(10), h.search(12), h.search(42), h.get(10))
    h.put(10,13); h.put(42,12); print(h, h.size, h.search(10), h.search(12), h.search(42), h.get(10))
    h.put("Algorithms","Are Great!"); print(h, h.get("Algorithms"))
    h.remove(10); print(h); h.get(10)

we should get the printout:

    {(10, 12)} 1 True False False 12
    {(10, 13),(42, 12)} 2 True False True 13
    {(10, 13),(42, 12),('Algorithms', 'Are Great!')} Are Great!
    {(42, 12),('Algorithms', 'Are Great!')}
    [...]
    Exception: ('HashMap key error:', 10)

In [7]:
class KVPair:
    def __init__(self, k, v=None):
        self.key = k
        self.val = v

    def __eq__(self, other):
        return self.key == other.key

    def __str__(self):
        return "("+str(self.key)+", "+str(self.val)+")" 
    
    def __repr__(self):
        return str((self.key,self.val))

class HashMap:
    def __init__(self, d=10):    # a default array length of 10
        self.inArray = [LinkedList() for i in range(d)] 
        self.size = 0
        self.threshold = 0.75    # a default threshold of 0.75

    def hash(self, d):
        return hash(d) % len(self.inArray)
        
    def __str__(self):
        def list2string(l):
            if l.length==0: return ""
            A = [None]*l.length
            ptr = l.head
            for i in range(len(A)):
                A[i] = ptr.data
                ptr = ptr.next
            return str(A)[1:-1]
        
        s = ""
        for i in range(len(self.inArray)):
            x = list2string(self.inArray[i])
            if x != "" and len(s)>1: s += ","+x
            else: s+=x 
        return "{"+s+"}"

    # The search function is almost identical to the one for HashTable.
    # We first find the slot where the pair with key k could be by hashing k.
    # We then search in the corresponding linked list for a pair with key k, 
    # by simply searching in it for the pair KVPair(k,None): this will work
    # because equality of KVPairs is determined by comparing their keys.
    def search(self, k):
        i = self.hash(k)
        j = self.inArray[i].search(KVPair(k)) 
        if j == -1: return False
        return True

    # insert in the map the pair (k,v)
    # if there exists a pair (k,v') "change it" to (k,v)
    def put(self, k, v):
        i = self.hash(k)
        j = self.inArray[i].search(KVPair(k)) 
        if j != -1:
            self.inArray[i].get(j).val = v
            return
        self.inArray[i].insert(0,KVPair(k,v))
        self.size += 1
        if self.size > self.threshold*len(self.inArray): self._resizeUp()
        
    def get(self, k):
        i = self.hash(k)
        j = self.inArray[i].search(KVPair(k)) 
        if j == -1: raise Exception("HashMap key error:",k)
        return self.inArray[i].get(j).val

    def remove(self, k):
        i = self.hash(k)
        if self.inArray[i].removeVal(KVPair(k)):
            self.size -= 1

    def _resizeUp(self):
        oldArray = self.inArray
        self.inArray = [LinkedList() for i in range(2*len(oldArray))]
        for i in range(len(oldArray)):
            while oldArray[i].length > 0:
                d = oldArray[i].remove(0)
                self.inArray[self.hash(d.key)].insert(0,d)
        
        
h = HashMap()
h.put(10,12); print(h, h.size, h.search(10), h.search(12), h.search(42), h.get(10))
h.put(10,13); h.put(42,12); print(h, h.size, h.search(10), h.search(12), h.search(42), h.get(10))
h.put("Algorithms","Are Great!"); print(h, h.get("Algorithms"))
h.remove(10); print(h); h.get(10)

{(10, 12)} 1 True False False 12
{(10, 13),(42, 12)} 2 True False True 13
{(10, 13),(42, 12),('Algorithms', 'Are Great!')} Are Great!
{(42, 12),('Algorithms', 'Are Great!')}


Exception: ('HashMap key error:', 10)

In [ ]:
# Question 3
A = [0, 34, 23, 4, 5, 12, 45, 42, 45, 10, 56, 90, 99, 10, 2]
h = HashTable(5)
for x in A: h.add(x)
myprint(h)

In [6]:
class Node:
    def __init__(self, d, n):
        self.data = d
        self.next = n

class LinkedList:
    def __init__(self):
        self.head = None
        self.length = 0

    def __str__(self):
        if self.head == None: 
            return "empty"
        st = "-"
        ptr = self.head
        while ptr != None:
            st += "-> "+str(ptr.data)+" "
            ptr = ptr.next
        return st+"|"
    
    def search(self, d):
        i = 0
        ptr = self.head
        while ptr != None:
            if ptr.data == d:
                return i
            ptr = ptr.next
            i += 1
        return -1
        
    def append(self, d):
        if self.head == None:      
            self.head = Node(d,None) 
        else:
            ptr = self.head
            while ptr.next != None:
                ptr = ptr.next
            ptr.next = Node(d,None)
        self.length += 1

    def insert(self, i, d):
        if self.head == None or i == 0:
            self.head = Node(d,self.head)
        else:
            ptr = self.head
            while i>1 and ptr.next != None:
                ptr = ptr.next
                i -= 1
            ptr.next = Node(d,ptr.next)
        self.length += 1

    def remove(self, i): # removes i-th element and returns it
        if self.head == None:
            return None
        if i == 0:
            val = self.head.data
            self.head = self.head.next
            self.length -= 1
            return val
        else:
            ptr = self.head
            while i>1 and ptr.next != None:
                ptr = ptr.next
                i -= 1
            if i == 1:
                val = ptr.next.data
                ptr.next = ptr.next.next
                self.length -= 1
                return val
            return None

    def get(self, i):
        ptr = self.head
        while ptr != None and i>0:
            ptr = ptr.next
            i -= 1
        return ptr.data
                
    # removes first occurrence of value d if found in the list
    # returns True if removed, False if not found
    def removeVal(self, d):
        if self.head == None: return False
        if self.head.data == d:
            self.head = self.head.next
            self.length -= 1
            return True
        ptr = self.head	
        while ptr.next != None:
            if ptr.next.data == d:
                ptr.next = ptr.next.next
                self.length -= 1
                return True
            ptr = ptr.next
        return False